# Lecture 12 – Relativistic kinematics

**Some exercises on relativistic kinematics using the `pylorentz` package**

Prepare the notebook with the preambles for the inclusion of pandas, numpy and matplotlib.pyplot:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Import the data file (csv format) into your Google Colab. Instead of mounting the "*/content/drive/MyDrive*" folder, there is an alternative way to download the data file to your Colab folder and access it directly. To do this,  install the **gdown** module of python, in the following way:

In [ ]:
!pip install gdown

Then you import the full gdown package and you pick the file you need. The most difficult thing is to file the url address of the file, which can be done inquiring the properties of the file through the web gdrive interface.

In [ ]:
!pip install gdown
import gdown

# now you use the url pointing to the data file
# the url is obtained from inquiring the file address in google drive

# real experimental data
# exp_nbarp-2pip1pim_93.csv
url = "https://drive.google.com/uc?id=17J4rrO-NHL8whkd7hjELhJbCoanoaqam"


# real data
# path = '/content/drive/MyDrive/Colab - Exp Meth. Hadron Spec /Data Files/Spectroscopy/exp_nbarp-2pip1pim_93.csv'
# montecarlo data
# gen_nbarp-2pip1pim_93.csv
# url = 'https://drive.google.com/uc?id=1SarwF44sWSGbpn4PmBH3GLKIJJmN2lbS'

output = "antineutron_3pi.dat"
gdown.download(url, output, quiet=False)
data = pd.read_csv(output)

The file contains the 4-momenta of the particles of the reaction $\bar n p \rightarrow \pi^+_1\pi^+_2\pi^-_3$, the last column corresponds to the 3rd component of the antineutron momentum (up to 300 MeV/c), which travels along the $z$ axis

The columns headers present some trailing blanks, that must be dropped to be able to use correctly the DataFormat structure (if not, they deliver an error message). To do so, the *str.strip()* function must be used beforehand to reformat the column shape.
In the following commands in the cell, columns are shown, overall with the data.columns command, and per single variable (like *data.p2x*). If the format is correct, no error should appear.

In [ ]:
data.columns = data.columns.str.strip()
data.head()

Load the **pylorentz package** (if not available, install it with *pip install pylorentz*).

The pylorentz package defines particle 4-momenta through the quantities

*   $E,\; \eta,\; \phi,\; p\;$ or
*   $E,\; \eta,\; \phi,\; p_T\;$
*   $E,\; m,\; \eta,\; \phi\;$
*   $m,\; \eta,\; \phi,\; p_T\;$
*   $m,\; \eta,\; \phi,\; p\;$

They are related to the cartesian coordinates of the particle 3-momentum through the following relationships:

> $p = \sqrt{p_x^2 + p_y^2 + p_z^2}$

> $p_T = \sqrt{p_x^2 + p_y^2}$

> $p_x = p_T\cos\phi = p\cos\phi\sin\theta$

> $p_y = p_T\sin\phi = p\sin\phi\sin\theta$

> $p_z = p_T\sinh\eta = p\cos\theta $

from which one can derive

> $\phi = \tan^{-1}(p_y/p_x)$

> $\theta = \cos^{-1}(p_z/p)$

> $\eta = -\ln(\tan(\theta/2)$

The pylorentz package provides functions for the evaluation of the magnitude of the 4-momentum vector, of the kinematic angles and their sum (which is convenient for particles systems).


  




In [ ]:
! pip install pylorentz

In [ ]:
import math

from pylorentz import Momentum4

Let's import the arrays from the csv table into Momentum4 objects and repeat the calculation of invariant masses and other observables. We are working with *numpy arrays* whose length is equale to the number of entries of the table read from the csv file.

In [ ]:
# final state
p1T = np.sqrt(data.p1x**2 + data.p1y**2)
p1mod = np.sqrt(p1T**2 + data.p1z**2)
eta1 = np.arcsinh(data.p1z / p1T)
phi1 = np.arctan2(data.p1y, data.p1x)
pion1 = Momentum4.e_eta_phi_p(data.E1, eta1, phi1, p1mod)

p2T = np.sqrt(data.p2x**2 + data.p2y**2)
p2mod = np.sqrt(p2T**2 + data.p2z**2)
eta2 = np.arcsinh(data.p2z / p2T)
phi2 = np.arctan2(data.p2y, data.p2x)
pion2 = Momentum4.e_eta_phi_p(data.E2, eta2, phi2, p2mod)

p3T = np.sqrt(data.p3x**2 + data.p3y**2)
p3mod = np.sqrt(p3T**2 + data.p3z**2)
eta3 = np.arcsinh(data.p3z / p3T)
phi3 = np.arctan2(data.p3y, data.p3x)
pion3 = Momentum4.e_eta_phi_p(data.E3, eta3, phi3, p3mod)

# initial state
# len is the number of events read by the csv file
len = data.pnbar.size
pnbT = np.zeros(len)
pnbmod = data.pnbar
etanb = 1.0e11 * np.ones(len)
phinb = np.zeros(len)
massNeutron = 0.93956
Enb = np.sqrt(massNeutron**2 + pnbmod**2)
antineutron = Momentum4.e_eta_phi_p(Enb, etanb, phinb, pnbmod)

# the target (proton) is at rest
massProton = 0.93827
ETgt = massProton * np.ones(len)
etaTgt = np.zeros(len)
phiTgt = np.zeros(len)
pTgt = np.zeros(len)
protonTarget = Momentum4.e_eta_phi_p(ETgt, etaTgt, phiTgt, pTgt)

In [ ]:
system12 = pion1 + pion2
system23 = pion2 + pion3
system13 = pion1 + pion3

invariantMassSquared12 = system12.m2
invariantMassSquared13 = system13.m2
invariantMassSquared23 = system23.m2

Let's plot the Dalitz Plots using the new Momentum4 objects. As in the first exercise let's plot the antineutron momentum to see how the distribution looks like.

In [ ]:
plt.hist(antineutron.p, bins=100, color="aquamarine", alpha=0.7)
plt.xlabel("Antineutron momentum [GeV/c]")
plt.ylabel("Entries/(4 MeV/c)")
plt.title("The histogram of the momentum of the incoming antineutron \n")

And now let's plot the two Dalitz plots with the square invariant masses of the pion pairs:

In [ ]:
fig, ax = plt.subplots(1, 2, tight_layout=True, figsize=(9, 4))
h0 = ax[0].hist2d(invariantMassSquared13, invariantMassSquared12, bins=100, cmap="jet")
h0 = ax[0].hist2d(invariantMassSquared23, invariantMassSquared12, bins=100, cmap="jet")
fig.colorbar(h0[3], ax=ax[0])
ax[0].set_xlabel("i.m.$^2(\pi^+_{(1,2)}\pi^-_{3}$) [GeV$^2$]")
ax[0].set_ylabel("i.m.$^2(\pi^+\pi^+$) [GeV$^2$]")
h1 = ax[1].hist2d(invariantMassSquared13, invariantMassSquared23, bins=100, cmap="jet")
h1 = ax[1].hist2d(invariantMassSquared23, invariantMassSquared13, bins=100, cmap="jet")
fig.colorbar(h1[3], ax=ax[1])
ax[1].set_xlabel("i.m.$^2(\pi^+_1\pi^-$) [GeV$^2$]")
ax[1].set_ylabel("i.m.$^2(\pi^+_2\pi^-$) [GeV$^2$]")

Let's transform the 4-momenta of the particles, which are defined in the lab system, into the center-of-mass system exploiting the *boostparticle* function of pylorentz:

In [ ]:
centerOfMass = antineutron + protonTarget

# to boost particles in the center of mass reference system, use the - sign
pion1CM = pion1.boost_particle(-centerOfMass)
pion2CM = pion2.boost_particle(-centerOfMass)
pion3CM = pion3.boost_particle(-centerOfMass)

In [ ]:
antineutronCM = antineutron.boost_particle(-centerOfMass)

In [ ]:
protonTargetCM = protonTarget.boost_particle(-centerOfMass)

Let's check whether the Mandelstam variables are indeed invariant in the different reference systems. Let's assume a two-body reaction $a + b\rightarrow c + d$ where in the final state two pions form a neutral dipion which recoils against the remaining $\pi^+_2$: $\bar n + p \rightarrow D(\pi^+_1\pi^-_3) + \pi^+_2$



> **total energy** $s = (p_a+p_b)^2 = (p_c + p_d)^2$



In [ ]:
s_lab = centerOfMass.m2
print(s_lab)

In [ ]:
initCM = antineutronCM + protonTargetCM
s_cm = initCM.m2
print(s_cm)

s_lab - s_cm

In [ ]:
plt.hist(s_lab, bins=100, color="MediumPurple")
plt.xlabel("s [(GeV/$c^2)^2$]")
plt.ylabel("Entries/(1.6 (MeV/$c^2)^2$)")
plt.title("Available energy for the reaction \n")




> Momentum transfer $t = (p_a - p_c)^2 = (p_b - p_d)^2$





In [ ]:
dipion = system13
t_lab_1 = (antineutron - dipion).m2
t_lab_2 = (protonTarget - pion2).m2
dipionCM = system13.boost_particle(-centerOfMass)
t_cm_1 = (antineutronCM - dipionCM).m2
t_cm_2 = (protonTargetCM - pion2CM).m2

In [ ]:
fig, ax = plt.subplots(2, 2, tight_layout=True, figsize=(9, 4))
ax[0, 0].hist(t_lab_1, bins=100, color="Lime")
ax[0, 0].set_xlabel("t = $(p_{nbar} - p_D)^2$ lab RF [$(\mathrm{GeV}/c^2)^2$]")
ax[0, 1].hist(t_lab_2, bins=100, color="LimeGreen")
ax[0, 1].set_xlabel("t = $(p_p - p_{\pi^+_2})^2$ lab RF [$(\mathrm{GeV}/c^2)^2$]")
ax[1, 0].hist(t_cm_1, bins=100, color="GreenYellow")
ax[1, 0].set_xlabel("t = $(p_{nbar} - p_D)^2$ CM RF [$(\mathrm{GeV}/c^2)^2$]")
ax[1, 1].hist(t_cm_2, bins=100, color="LawnGreen")
ax[1, 1].set_xlabel("t = $(p_p - p_{\pi^+_2})^2$ CM RF [$(\mathrm{GeV}/c^2)^2$]")

> $u = (p_a - p_d)^2 = (p_b - p_c)^2$


In [ ]:
u_lab_1 = (antineutron - pion2).m2
u_lab_2 = (protonTarget - dipion).m2
u_cm_1 = (antineutronCM - pion2CM).m2
u_cm_2 = (protonTargetCM - dipionCM).m2

In [ ]:
fig, ax = plt.subplots(2, 2, tight_layout=True, figsize=(9, 4))
ax[0, 0].hist(u_lab_1, bins=100, color="HotPink")
ax[0, 0].set_xlabel("u = $(p_{nbar} - p_{\pi^+_2})^2$ lab RF [$(\mathrm{GeV}/c^2)^2$]")
ax[0, 1].hist(u_lab_2, bins=100, color="DeepPink")
ax[0, 1].set_xlabel("u = $(p_p - p_D)^2$ lab RF [$(\mathrm{GeV}/c^2)^2$]")
ax[1, 0].hist(u_cm_1, bins=100, color="Fuchsia")
ax[1, 0].set_xlabel("u = $(p_{nbar} - p_{\pi^+_2})^2$ CM RF [$(\mathrm{GeV}/c^2)^2$]")
ax[1, 1].hist(u_cm_2, bins=100, color="DarkMagenta")
ax[1, 1].set_xlabel("u = $(p_p - p_D)^2$ CM RF [$(\mathrm{GeV}/c^2)^2$]")

Test: $\; s + t + u = m_1^2 + m_2^2 + m_3^2 + m_4^2$

In [ ]:
sum = s_lab + t_lab_1 + u_lab_1

In [ ]:
sumOfMasses = (massNeutron**2 + massProton**2 + dipionCM.m2 + pion2CM.m2) * np.ones(
    len
)

In [ ]:
plt.hist(sum - sumOfMasses, bins=100, color="CornFlowerBlue")

Another interesting quantity to be observed is the **missing mass**. It can be evaluated comparing the total energies between the initial and the final state of a reaction, or, given a final state, evaluating the missing energy recoiling against a system of particles. In any case, the missing mass quantity is defined as the modulus of the 4-momentum corresponing to the difference of the 4-momenta of the involved particles.

In [ ]:
# missing mass between initial and final state
initialState4Mom = centerOfMass
finalState4Mom = pion1 + pion2 + pion3
missing4Momentum = initialState4Mom - finalState4Mom
# missing mass recoiling against the neutral dipion systems (there are two)
dipion1 = pion1 + pion3
dipion2 = pion2 + pion3
recoilingMissingMass1 = finalState4Mom - dipion1
recoilingMissingMass2 = finalState4Mom - dipion2

Let's plot the missing mass of the reaction, the scatter plot of total momentum versus total energy of the measured particles in the final state (it should be close to zero), and the missing mass recoiling against the neutral dipion (it should be close to the mass of a pion, for a correctly selected exclusive reaction).

In [ ]:
fig, ax = plt.subplots(1, 3, tight_layout=True, figsize=(11, 3.5))
ax[0].hist(missing4Momentum.m2, bins=100, color="crimson")
ax[0].set_xlabel("m.m. [$(\mathrm{GeV}/c^2)^2$]")
# ptot vs Etot plot
h1 = ax[1].hist2d(finalState4Mom.e, finalState4Mom.p, bins=100, cmap="rainbow")
fig.colorbar(h1[3], ax=ax[1])
ax[1].set_xlabel("$\mathrm{E}_{tot}(2\pi^+\pi^-)$ [GeV]")
ax[1].set_ylabel("$\mathrm{p}_{tot}(2\pi^+\pi^-)$ [GeV/c]")
# missing mass recoiling against the neutral dipion (2 entries/event)
allMissingDipion = np.concatenate([recoilingMissingMass1.m2, recoilingMissingMass2.m2])
missingHisto = ax[2].hist(allMissingDipion, bins=100, color="darkorange")
ax[2].set_xlabel("m.m. recoiling against $(\pi^+\pi^-)$ [$(\mathrm{GeV}/c^2)^2$]")
ax[2].xaxis.get_label().set_fontsize(8)
ax[2].tick_params(axis="both", which="major", labelsize=8)

Let's fit the last plot with a gaussian function to check if the missing particle is really a charged pion (mass: 0.140 GeV/$c^2$).

In [ ]:
import matplotlib.mlab as mlab
from scipy.stats import norm

# best fit of data
(mu, sigma) = norm.fit(allMissingDipion)

# the histogram of the data
n, bins, patches = plt.hist(
    allMissingDipion,
    range=(0.01947835, 0.01947985),
    density=True,
    facecolor="darkorange",
    histtype="barstacked",
)

# add a 'best fit' line
y = norm.pdf(bins, mu, sigma)
l = plt.plot(bins, y, "r--", linewidth=2)

# plot
plt.xlabel("missing mass recoiling against the neutral dipion")
plt.ylabel("counts")
plt.title(r"$\mu=%.3f,\ \sigma=%.3f$" % (mu, sigma))
plt.grid(True)
plt.hist(allMissingDipion, 100, density=True, facecolor="orange", histtype="barstacked")

print("The missing mass in GeV/c^2 is: ", np.sqrt(mu))